In [49]:
from __future__ import division
import six
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types
import re
import numpy as np
import csv
import pandas as pd
import sys

In [62]:
GOOGLE_APPLICATION_CREDENTIALS="My First Project-cd24ed9156c8.json"

In [4]:
def clean_tweet(tweet): 
    ''' 
    Utility function to clean tweet text by removing links, special characters 
    using simple regex statements. 
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

In [7]:
def get_tweets(handle): 
    ''' 
    Main function to fetch tweets and parse them. 
    '''
    # eppty list to store parsed tweets 
    with open(handle + '_tweets.csv', 'r') as f:
        reader = csv.reader(f)
        tweets = list(reader)
          
    fetched_tweets = []
    for i in np.arange(1,len(tweets)):
        fetched_tweets.append(tweets[i][2])
        
    return fetched_tweets

In [9]:
tweets = get_tweets('BorisJohnson')

In [18]:
def word_in_text(word, text):
    word = word.lower()
    text = text.lower()
    match = re.search(word, text)
    if match:
        return True
    return False

def word_basket_in_text(word_list, text):
    status = True
    for word in word_list:
        status = status and word_in_text(word, text)
    return status

In [35]:
Healthcare = [u'care','affordable care act', 'medicare', 'obamacare',  'benefits', 'opioid', 'abortion', 'medicaid', 'single payer', 'medication', 'prescription', 'drugs', 'hospital', 'health', 'care']
Terrorism = [u'al', u'terror', u'sanctions', u'drone', u'oil', u'gulf', u'intelligence', u'ISIS', u'military', u'nuclear']
Economy = [u'china', u'tariff', u'mexico', u'fed', u'sec', u'interest', u'rate', u'immigration', u'debt', u'unemployment', u'growth', u'inflation', u'trade war', u'dollar', u'gdp', u'g20', u'recession', u'stock', u'market']
lgbtq = [u'gay', u'lesbian', u'pride', u'community', u'lgbt', u'same-sex', u'marriage', u'orientation', u'parade', u'stonewall', u'riot', u'civil', u'homosexual', u'queer', u'conversion']

In [36]:
BJ_healthcare = []
for tweet in tweets:
    for key in Healthcare:
        if word_in_text(key, tweet):
            BJ_healthcare.append(clean_tweet(tweet))

In [37]:
BJ_healthcare = list(set(BJ_healthcare))

In [45]:
def implicit():
    from google.cloud import storage

    # If you don't specify credentials when constructing the client, the
    # client library will look for credentials in the environment.
    storage_client = storage.Client()

    # Make an authenticated API request
    buckets = list(storage_client.list_buckets())
    print(buckets)

'Delighted to meet Simon who has spent two years researching developing and formulating his products for UXB Skincare Each one is now made by hand in Uxbridge It s this passion and dedication that makes small businesses stand out SmallBusinessSaturday ShopLocal'

In [82]:
client = language.LanguageServiceClient.from_service_account_json(GOOGLE_APPLICATION_CREDENTIALS)

def sentiment_analyzer(list_of_tweets, list_of_keywords): 
    for text in list_of_tweets:
        if isinstance(text, six.binary_type):
            text = text.decode('utf-8')

        document = types.Document(
            content=text.encode('utf-8'),
            type=enums.Document.Type.PLAIN_TEXT)

        # Detect and send native Python encoding to receive correct word offsets.
        encoding = enums.EncodingType.UTF32
        if sys.maxunicode == 65535:
            encoding = enums.EncodingType.UTF16

        result = client.analyze_entity_sentiment(document, encoding)
        print(text)
        for entity in result.entities:
            entity_name = entity.name
            if entity_name in list_of_keywords:
                print(u'Name: "{}"'.format(entity.name))
                for mention in entity.mentions:
                    print(u'  Begin Offset : {}'.format(mention.text.begin_offset))
                    print(u'  Content : {}'.format(mention.text.content))
                    print(u'  Magnitude : {}'.format(mention.sentiment.magnitude))
                    print(u'  Sentiment : {}'.format(mention.sentiment.score))
                    print(u'  Type : {}'.format(mention.type))
                print(u'Salience: {}'.format(entity.salience))
                print(u'Sentiment: {}\n'.format(entity.sentiment))
        print('--------')

In [83]:
sentiment_analyzer(BJ_healthcare, Healthcare)

RT The British people won t be scared into backing a woeful Brexit deal nobody voted for
--------
Wonderful that Yulia Skripal is out of hospital and here s to her full and speedy recovery Thanks again to the incredible NHS staff in Salisbury I wish Sergei Skripal all the best and hope he recuperates quickly too
Name: "hospital"
  Begin Offset : 39
  Content : hospital
  Magnitude : 0.0
  Sentiment : 0.0
  Type : 2
Salience: 0.07005702704191208
Sentiment: 

--------
Sergei Skripal s improved condition is both great news and testament to the quick work of our world class emergency services and NHS staff who have been paramount to his and Yulia s recovery I thank all involved and wish Sergei amp Yulia a quick return to full health
Name: "health"
  Begin Offset : 261
  Content : health
  Magnitude : 0.5
  Sentiment : 0.5
  Type : 2
Salience: 0.017671002075076103
Sentiment: magnitude: 0.5
score: 0.5


--------
A real pleasure to welcome Prime Minister to the UK Lots to discuss on bilateral